In [1]:
include("../src/Julia.jl")

pcgWorker (generic function with 1 method)

In [2]:
pwd()

"/Users/serbanstan/git/TreePCG/julia/testing"

In [3]:
graphp = "../../graphs/randdense_iu_1000_1/"

"../../graphs/randdense_iu_1000_1/"

In [4]:
a = readFromFile(graphp * "graph.mtx");

In [5]:
tree = []
for i in 1:5
    tr = readFromFile(graphp * "tree$(i).mtx");
    push!(tree, tr);
end

In [6]:
x = readFromFile(graphp * "x.vec");

In [7]:
la = lap(a);
x = x - mean(x)
b = la * x; b = b - mean(b);

In [8]:
numIts = 200;

@time for i in 4:5
    @time F = treeSolver(tree[i])
    @time F(b);
    
    # a run on standard Float64 data types
    @time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=false);
    norm(lap(a) * myx64 - b) / norm(b)
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_tree$(i)_default.txt", dbg)
    
    # now useing higer precision data types
    for hp in [64,128,256,512,1024]
        set_bigfloat_precision(hp)
        laHP = toHighPrecision(la; precision=hp)
        treeHP = toHighPrecision(tree[i]; precision=hp)
        xHP = toHighPrecision(x; precision=hp);
        bHP = toHighPrecision(b; precision=hp);
        FHP = treeSolver(treeHP)

        @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
        insert!(dbg, 1, "$(numIts) iterations")
        insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
        writeToFile(graphp * "_log_julia_tree$(i)_$(hp).txt", dbg)
    end
end

  1.120251 seconds (275.40 k allocations: 11.699 MB, 1.48% gc time)
  0.356544 seconds (123.92 k allocations: 4.987 MB)
  3.095252 seconds (3.17 M allocations: 95.356 MB, 1.75% gc time)
Working on iteration 10 with a-norm error 27.354695604520916
Working on iteration 20 with a-norm error 10.23209383952411
Working on iteration 30 with a-norm error 4.075423256586777
Working on iteration 40 with a-norm error 1.6444117507868967
Working on iteration 50 with a-norm error 0.7292394559136794
Working on iteration 60 with a-norm error 0.33214043909133995
Working on iteration 70 with a-norm error 0.1344380095841585
Working on iteration 80 with a-norm error 0.06159149056051112
Working on iteration 90 with a-norm error 0.024477125554235667
Working on iteration 100 with a-norm error 0.009857500181620224
Working on iteration 110 with a-norm error 0.004353345333663311
Working on iteration 120 with a-norm error 0.0014949862315196726
Working on iteration 130 with a-norm error 0.0006711786516010059
Worki